In [ ]:
import warnings
warnings.filterwarnings('ignore')

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_absolute_percentage_error
from pathlib import Path

In [ ]:
def create_lags(df, lags_list, target_col='valor', window_size=4):
    """
    Cria defasagens (lags), features de janela móvel, e features de tempo/sazonalidade
    (Ano, Mês e componentes Seno/Cosseno) para a série temporal.

    df (DataFrame): O DataFrame original com a série temporal (índice datetime).
    lags_list (list): Lista de defasagens a serem criadas (ex: [1, 52]).
    target_col (str): Nome da coluna alvo (y_t).
    window_size (int): Tamanho da janela para as estatísticas móveis.
    """
    # DataFrame auxiliar para armazenar as features.
    df_features = pd.DataFrame(index=df.index)

    # 1. Variável Alvo (y_t): O valor da série no momento t.
    df_features['y'] = df[target_col].copy()

    # --- NOVO BLOCO DE FEATURE ENGINEERING: FEATURES DE TEMPO E SAZONALIDADE ---

    # Extrai features simples de calendário
    df_features['time_index'] = np.arange(len(df_features)) + 1
    # Adicione esta linha junto com a criação do 'time_index'
    df_features['time_index_sq'] = df_features['time_index'] ** 2
    df_features['ano'] = df.index.year
    df_features['mes'] = df.index.month
    df_features['semana_do_ano'] = df.index.isocalendar().week.astype(int)

    # Cria features de sazonalidade cíclica (Seno/Cosseno)
    # Máximo de semanas em um ano = 52
    period = 52.0
    df_features['sin_semana'] = np.sin(2 * np.pi * df_features['semana_do_ano'] / period)
    df_features['cos_semana'] = np.cos(2 * np.pi * df_features['semana_do_ano'] / period)

    # Remove colunas intermediárias (semana_do_ano não é mais necessária)
    df_features.drop('semana_do_ano', axis=1, inplace=True)

    # --- FIM DO NOVO BLOCO ---

    # 2. Features de Defasagem (X): Valores da série em momentos passados (y_{t-k}).
    for lag in lags_list:
        df_features[f'lag_{lag}'] = df[target_col].shift(lag)

    # 3. Features de Janela Móvel (Rolling Windows)
    # Usamos shift(1) para garantir que a janela móvel só use dados do passado.
    rolling_window = df[target_col].shift(1).rolling(window=window_size)

    df_features[f'rolling_mean_{window_size}'] = rolling_window.mean()
    df_features[f'rolling_std_{window_size}'] = rolling_window.std()
    df_features[f'rolling_max_{window_size}'] = rolling_window.max()
    df_features[f'rolling_min_{window_size}'] = rolling_window.min()

    # Remove as linhas iniciais que contêm valores NaN (devido aos lags e janelas).
    df_features.dropna(inplace=True)

    # X é a matriz de features, Y é a variável alvo.
    X = df_features.drop('y', axis=1)
    Y = df_features['y']

    return X, Y


In [ ]:
CAMINHO_RAIZ = Path(os.getcwd()).resolve().parent.parent
OUTPUT = CAMINHO_RAIZ / 'out' / 'dengue_pernambuco'
os.makedirs(OUTPUT, exist_ok=True)

dengue_pe = pd.read_excel(CAMINHO_RAIZ / 'data' / 'dengue_pernambuco.xlsx')
print('Dimensões do dataset:', dengue_pe.shape)
print('Colunas:', dengue_pe.columns.tolist())
display(dengue_pe.head(8))

In [ ]:
dengue_pe['semana'] = pd.to_datetime(dengue_pe['semana'])
plt.figure(figsize=(12,3))
plt.plot(dengue_pe['semana'], dengue_pe['valor'])
plt.title('Série temporal de casos')
plt.xlabel('semana')
plt.ylabel('valor')
plt.tight_layout()
plt.show()

plt.figure(figsize=(6,4))
sns.histplot(dengue_pe['valor'], bins=60, kde=False)
plt.title('Distribuição de valores')
plt.show()
dengue_pe.set_index('semana', inplace=True)
dengue_pe.sort_index(inplace=True)

In [ ]:
n_obs = len(dengue_pe)
p_train = 0.50
p_val = 0.25
n_train = int(n_obs * p_train)
n_val = int(n_obs * p_val)
n_test = n_obs - n_train - n_val

dengue_train = dengue_pe.iloc[:n_train]
dengue_val = dengue_pe.iloc[n_train: n_train + n_val]
dengue_test = dengue_pe.iloc[n_train + n_val:]

In [ ]:
LAG_FEATURES = [1, 2, 3, 4, 12, 52]

# 2. Aplicamos a mesma função de antes, mas agora passamos o 'window_size'.
# A função irá criar automaticamente as features de média, desvio padrão, etc.
X_full, Y_full = create_lags(dengue_pe, LAG_FEATURES, window_size=4)


# Exibimos as primeiras linhas para verificar as NOVAS features.
print("Estrutura do Conjunto de Features e Alvo (X e Y):")
print("Novas colunas em X:", X_full.columns.tolist())
print("\n")
print(X_full.head())

In [ ]:
# 1. Definição das colunas que recebem np.log1p
COLS_TO_TRANSFORM = [col for col in X_full.columns if 'lag_' in col or 'rolling_' in col]

train_val_end_date = dengue_val.index.max()
train_end_date = dengue_train.index.max()

# --- Separação com Transformação Seletiva ---

# TREINO
X_train = X_full.loc[:train_end_date].copy()
X_train[COLS_TO_TRANSFORM] = np.log1p(X_train[COLS_TO_TRANSFORM])
Y_train = np.log1p(Y_full.loc[:train_end_date])

# VALIDAÇÃO
X_val = X_full.loc[train_end_date:train_val_end_date].iloc[1:].copy()
X_val[COLS_TO_TRANSFORM] = np.log1p(X_val[COLS_TO_TRANSFORM])
Y_val = np.log1p(Y_full.loc[train_end_date:train_val_end_date].iloc[1:])

# TESTE
X_test = X_full.loc[train_val_end_date:].iloc[1:].copy()
X_test[COLS_TO_TRANSFORM] = np.log1p(X_test[COLS_TO_TRANSFORM])
Y_test = np.log1p(Y_full.loc[train_val_end_date:].iloc[1:])

# --- Limpeza Final (Crucial para NaN/Inf) ---

# 1. Recriação do conjunto Treino/Validação (a ser otimizado)
X_train_val = pd.concat([X_train, X_val])
Y_train_val = pd.concat([Y_train, Y_val])

# 2. Limpeza Agressiva
X_train_val.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)

# 3. Imputação com a média de X_train_val
train_val_mean = X_train_val.mean()
X_train_val.fillna(train_val_mean, inplace=True)
X_test.fillna(train_val_mean, inplace=True)

# 4. Conversão para float64 (Solução para o erro persistente de dtype)
X_train_val = X_train_val.astype(np.float64)
X_test = X_test.astype(np.float64)

# Verificação:
print(f"time_index após separação (deve ser linear): {X_train_val['time_index'].head()}")

In [ ]:
def reverse_log_mape_scorer(Y_true, Y_pred):
    """Reverte a transformação logarítmica (np.expm1) para calcular o MAPE em casos reais."""
    # A métrica usa np.expm1() para converter os valores logarítmicos de volta à escala original (casos de dengue).
    return mean_absolute_percentage_error(np.expm1(Y_true), np.expm1(Y_pred))

# Criamos o scorer para ser usado no RandomizedSearchCV.
# greater_is_better=False indica que o MAPE deve ser MINIMIZADO.
mape_negativo = make_scorer(reverse_log_mape_scorer, greater_is_better=False)

# --- 2. EXECUÇÃO DO RANDOMIZED SEARCH (AJUSTADA) ---

# Define o espaço de busca para o Random Forest. (MANTIDO O SEU)
param_dist = {
    'n_estimators': [200, 400, 600],
    'max_depth': [5, 10, 15, 20, None],
    'min_samples_leaf': [2, 5, 10],
    'max_features': ['sqrt', 'log2']
}

# TimeSeriesSplit (MANTIDO O SEU)
tscv = TimeSeriesSplit(n_splits=5)

# RandomizedSearchCV: Realiza a busca aleatória.
random_search = RandomizedSearchCV(
    estimator=RandomForestRegressor(random_state=42, n_jobs=-1), # Adicionado n_jobs=-1 para paralelismo
    param_distributions=param_dist,
    n_iter=30, # Testaremos 20 combinações
    scoring=mape_negativo, # CORREÇÃO: Usando o scorer com reversão logarítmica
    cv=tscv,
    random_state=42,
    verbose=2,
    n_jobs=-1
)

# O método fit() executa a busca no conjunto Treinamento + Validação (X_train_val).
random_search.fit(X_train_val, Y_train_val)

# best_params_ armazena os hiperparâmetros que obtiveram o melhor score (menor MAPE) na validação cruzada.
best_params_ap = random_search.best_params_
best_score_ap = -random_search.best_score_ # Reverte o score para o MAPE positivo.

print("\n--- Modelo AP (Random Forest) Selecionado ---")
print(f"Melhores Parâmetros (Random Search): {best_params_ap}")
print(f"Melhor MAPE na Validação Cruzada: {best_score_ap:.2%}")

In [ ]:
# --- TREINAMENTO FINAL (CÉLULA COM O ERRO) ---

# Instancia o Random Forest com os melhores parâmetros encontrados.
# (Assumindo que best_params_ap já foi definido pelo RandomizedSearchCV)
rf_ap_model = RandomForestRegressor(**best_params_ap, random_state=42, n_jobs=-1)

# Treinamento final no conjunto X_train_val (agora totalmente float64).
rf_ap_model.fit(X_train_val, Y_train_val)

# Previsões: ESTE BLOCO DEVE RODAR SEM ERRO AGORA
y_pred_train_ap = rf_ap_model.predict(X_train)
y_pred_val_ap = rf_ap_model.predict(X_val)
y_pred_test_ap = rf_ap_model.predict(X_test)

In [ ]:
plt.figure(figsize=(12,6))

# Linha real contínua
plt.plot(pd.concat([Y_train_val, Y_val, Y_test]),
         label="Real", color="black", linewidth=2)

# Previsões destacadas por fase
plt.plot(pd.Series(y_pred_train_ap, index=Y_train_val.index[-len(y_pred_train_ap):]),
         label="Previsto (Treino)", color="blue", linestyle="--")

plt.plot(pd.Series(y_pred_val_ap, index=Y_val.index),
         label="Previsto (Validação)", color="orange", linestyle="--")

plt.plot(pd.Series(y_pred_test_ap, index=Y_test.index),
         label="Previsto (Teste)", color="red", linestyle="--")

plt.title("Valores Reais vs Previstos - Treino, Validação e Teste")
plt.xlabel("Tempo")
plt.ylabel("Valor")
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig(OUTPUT / 'Random Florest - Valores Reais vs Previstos - Treino, Validação e Teste.png')
plt.show()


In [ ]:
# O modelo já está treinado e otimizado (rf_ap_model)
# Note: X_train_val é o conjunto com as colunas de features
feature_importances = pd.Series(
    rf_ap_model.feature_importances_,
    index=X_train_val.columns
).sort_values(ascending=False)

# Top 5 Features:
print("\nAs 5 features mais importantes são:")
print(feature_importances.head(5))

# Plot da Importância das Features
plt.figure(figsize=(10, 6))
sns.barplot(x=feature_importances.values, y=feature_importances.index, color='skyblue')
plt.title('Importância das Features (Random Forest)')
plt.xlabel('Score de Importância')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()

In [ ]:
# Métrica de Erro: Cálculo do EQM e MAPE
def evaluate_metrics(y_true, y_pred, name):
    """Calcula e retorna EQM e MAPE para um conjunto específico."""
    mse = mean_squared_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    return {'Conjunto': name, 'EQM': mse, 'MAPE': mape}

results_ap = [
    evaluate_metrics(Y_train, y_pred_train_ap, 'Treinamento (AP)'),
    evaluate_metrics(Y_val, y_pred_val_ap, 'Validação (AP)'),
    evaluate_metrics(Y_test, y_pred_test_ap, 'Teste (AP)')
]
df = pd.DataFrame(results_ap)
for val in results_ap:
    print(val)

# Gráfico de barras lado a lado
fig, ax1 = plt.subplots(1,2, figsize=(12,5))

# EQM
ax1[0].bar(df['Conjunto'], df['EQM'], color=['blue','orange','red'])
ax1[0].set_title("EQM (Erro Quadrático Médio)")
ax1[0].set_ylabel("Valor")

# MAPE
ax1[1].bar(df['Conjunto'], df['MAPE'], color=['blue','orange','red'])
ax1[1].set_title("MAPE (Erro Percentual Absoluto Médio)")
ax1[1].set_ylabel("Valor")

plt.suptitle("Comparação de Métricas por Conjunto")
plt.savefig(OUTPUT / 'Random Florest - Comparação de Métricas por Conjunto.png')
plt.show()


In [ ]:
residuos = Y_test - y_pred_test_ap

plt.figure(figsize=(8,5))
plt.hist(residuos, bins=30, color="skyblue", edgecolor="black")
plt.title("Distribuição dos Erros")
plt.xlabel("Erro")
plt.ylabel("Frequência")
plt.savefig(OUTPUT / 'Random Florest - Distribuição dos Erros.png')
plt.show()